In [1]:
%pip install -r requirements.txt

In [2]:
import numpy as np

import pandas as pd

from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

In [52]:
df = pd.read_csv('df_2.csv', index_col=0)

In [53]:
# df['min_park_dist'] = df[['park_eko', 'attraction', 'south', 'eternal_fire', 'veter']].min(axis=1)
# df = df.drop(['park_eko', 'attraction', 'south', 'eternal_fire', 'veter'])
df = df.drop(['date', 'day', 'month', 'year', 'dayofweek', 'street', 'lat_long', 'price_sq_meter'], axis=1)


In [78]:
df.iloc[:2].to_json(force_ascii=True)

'{"rooms":{"0":2,"1":2},"district":{"0":"nan","1":"\\u041e\\u0440\\u0434\\u0436\\u043e\\u043d\\u0438\\u043a\\u0438\\u0434\\u0437\\u0435\\u0432\\u0441\\u043a\\u0438\\u0439"},"floor":{"0":"5.0","1":"13.0"},"total_floors":{"0":"9.0","1":"14.0"},"total_area":{"0":49.8,"1":66.2},"living_area":{"0":30.0,"1":40.0},"kitchen_area":{"0":9.0,"1":10.5},"price":{"0":4000.0,"1":3999.0},"park_eko":{"0":null,"1":5.49},"attraction":{"0":null,"1":3.89},"south":{"0":null,"1":1.61},"eternal_fire":{"0":null,"1":4.77},"veter":{"0":null,"1":7.27},"city_center":{"0":null,"1":5.08},"floor_type":{"0":"middle","1":"middle"}}'

In [72]:
df

,rooms,district,floor,total_floors,total_area,living_area,kitchen_area,price,park_eko,attraction,south,eternal_fire,veter,city_center,floor_type
0,2,nan,5.0,9.0,49.8,30.0,9.0,4000.0,NaN,NaN,NaN,NaN,NaN,NaN,middle
1,2,Орджоникидзевский,13.0,14.0,66.2,40.0,10.5,3999.0,5.49,3.89,1.61,4.77,7.27,5.08,middle
2,3,Орджоникидзевский,7.0,10.0,65.0,46.0,9.0,5400.0,5.72,3.45,1.33,6.16,8.81,6.18,middle
3,3,Ленинский,5.0,5.0,62.4,43.0,8.0,4500.0,2.63,4.23,6.86,0.88,1.94,0.85,last
4,1,Ленинский,4.0,4.0,32.4,17.0,6.0,2300.0,4.14,6.30,9.34,3.38,1.50,3.16,last
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,3,Ленинский,5.0,5.0,74.0,56.0,9.0,5500.0,4.32,6.22,8.82,2.81,0.34,2.86,last
461,2,Ленинский,1.0,5.0,41.0,22.2,7.2,930.0,12.63,14.30,15.76,10.60,8.28,10.96,first
462,2,Правобережный,5.0,5.0,42.5,29.0,6.0,2650.0,1.74,3.41,6.40,1.03,2.77,0.31,last
463,2,Орджоникидзевский,3.0,9.0,50.0,30.0,9.0,3350.0,5.87,4.29,1.65,5.06,7.53,5.41,middle


In [55]:
df.isin([None]).sum()

rooms           0
district        0
floor           0
total_floors    0
total_area      0
living_area     0
kitchen_area    0
price           0
park_eko        0
attraction      0
south           0
eternal_fire    0
veter           0
city_center     0
floor_type      0
dtype: int64

In [57]:
categorical = ['district', 'floor', 'total_floors', 'floor_type']

df[categorical] = df[categorical].astype(str)

In [58]:
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=47,
                                                    shuffle=True,
                                                    #stratify=df['price']
                                                    )

In [59]:
print(f"Категориальные признаки: {categorical}")

numerical = list(set(X_train.columns) - set(categorical))
print(f"Числовые признаки: {numerical}")

Категориальные признаки: ['district', 'floor', 'total_floors', 'floor_type']
Числовые признаки: ['eternal_fire', 'total_area', 'south', 'kitchen_area', 'park_eko', 'veter', 'city_center', 'rooms', 'living_area', 'attraction']


In [60]:
pipe = Pipeline(
    [
        ("regressor", CatBoostRegressor())
    ]
)

param_grid = [
    {
        'regressor': [CatBoostRegressor(random_state=47, 
                                        eval_metric='RMSE',
                                        verbose=100,
                                        #silent=True,
                                        cat_features=categorical,
                                        early_stopping_rounds=20)],
        'regressor__depth': range(4, 12, 2),
        'regressor__learning_rate': np.linspace(0.01, 0.09, 7),
        'regressor__iterations': range(200, 1200, 200)
    }
]

grid = RandomizedSearchCV(pipe, param_grid, 
                          n_iter=10 ,
                          cv=3, 
                          scoring = 'neg_mean_squared_error', 
                          verbose=1)

In [61]:
%%time
grid.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


0:	learn: 1387.8321670	total: 75.3ms	remaining: 1m 15s
100:	learn: 427.8762557	total: 1.91s	remaining: 17s
200:	learn: 263.2375670	total: 4.52s	remaining: 18s
300:	learn: 176.3403577	total: 6.59s	remaining: 15.3s
400:	learn: 127.9303496	total: 8.81s	remaining: 13.2s
500:	learn: 97.6786237	total: 10.7s	remaining: 10.7s
600:	learn: 76.7002112	total: 12.6s	remaining: 8.4s
700:	learn: 58.9928681	total: 14.5s	remaining: 6.18s
800:	learn: 48.1498372	total: 16.4s	remaining: 4.07s
900:	learn: 39.1272297	total: 18.3s	remaining: 2.01s
999:	learn: 32.6356841	total: 20.6s	remaining: 0us
0:	learn: 1554.4855289	total: 16.4ms	remaining: 16.4s
100:	learn: 448.9345148	total: 2.28s	remaining: 20.3s
200:	learn: 298.3806179	total: 4.18s	remaining: 16.6s
300:	learn: 201.0748635	total: 6.04s	remaining: 14s
400:	learn: 142.0420628	total: 8.36s	remaining: 12.5s
500:	learn: 105.9649046	total: 10.5s	remaining: 10.5s
600:	learn: 79.9094870	total: 12.3s	remaining: 8.19s
700:	learn: 61.6153019	total: 14.3s	remaini

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('regressor',
                                              <catboost.core.CatBoostRegressor object at 0x0000022F4F910CA0>)]),
                   param_distributions=[{'regressor': [<catboost.core.CatBoostRegressor object at 0x0000022F4F9110C0>],
                                         'regressor__depth': range(4, 12, 2),
                                         'regressor__iterations': range(200, 1200, 200),
                                         'regressor__learning_rate': array([0.01      , 0.02333333, 0.03666667, 0.05      , 0.06333333,
       0.07666667, 0.09      ])}],
                   scoring='neg_mean_squared_error', verbose=1)

In [62]:
result = pd.DataFrame(grid.cv_results_)
result[['split0_test_score','split1_test_score','split2_test_score','mean_test_score','std_test_score']] = \
(result[['split0_test_score','split1_test_score','split2_test_score','mean_test_score','std_test_score']]*-1)**0.5

In [63]:
result.sort_values('mean_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__learning_rate,param_regressor__iterations,param_regressor__depth,param_regressor,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
6,10.902446,0.095686,0.003002,1.573482e-06,0.09,600,6,<catboost.core.CatBoostRegressor object at 0x0...,"{'regressor__learning_rate': 0.09, 'regressor_...",1101.061128,713.797674,899.876246,918.625783,NaN,1
0,20.639137,1.844969,0.004002,1.413830e-03,0.09,1000,6,<catboost.core.CatBoostRegressor object at 0x0...,"{'regressor__learning_rate': 0.09, 'regressor_...",1102.427564,715.262683,900.288303,919.685865,NaN,2
3,17.568792,0.063490,0.003333,4.709792e-04,0.09,600,8,<catboost.core.CatBoostRegressor object at 0x0...,"{'regressor__learning_rate': 0.09, 'regressor_...",1112.785800,718.937108,907.960439,927.281863,NaN,3
7,9.166378,0.074286,0.003673,4.766008e-04,0.036667,800,4,<catboost.core.CatBoostRegressor object at 0x0...,{'regressor__learning_rate': 0.036666666666666...,1048.489060,707.378397,991.544296,927.878152,NaN,4
9,31.385876,2.348036,0.003667,4.710895e-04,0.023333,1000,8,<catboost.core.CatBoostRegressor object at 0x0...,{'regressor__learning_rate': 0.023333333333333...,1130.569963,711.973277,937.798070,942.436489,NaN,5
4,22.105048,1.469353,0.003001,7.370010e-07,0.01,800,8,<catboost.core.CatBoostRegressor object at 0x0...,"{'regressor__learning_rate': 0.01, 'regressor_...",1129.355219,726.202150,949.873007,949.573852,NaN,6
2,35.873364,1.529871,0.004665,4.717638e-04,0.036667,800,10,<catboost.core.CatBoostRegressor object at 0x0...,{'regressor__learning_rate': 0.036666666666666...,1163.754171,705.740281,938.398457,954.460542,NaN,7
8,17.303835,0.499827,0.003332,4.712580e-04,0.063333,400,10,<catboost.core.CatBoostRegressor object at 0x0...,{'regressor__learning_rate': 0.063333333333333...,1160.823649,718.757529,945.394857,958.800530,NaN,8
5,2.351038,0.090777,0.001999,2.247832e-07,0.023333,200,4,<catboost.core.CatBoostRegressor object at 0x0...,{'regressor__learning_rate': 0.023333333333333...,1122.715686,737.435080,986.138189,962.075800,NaN,9
1,8.434183,0.342995,0.193626,2.695863e-01,0.09,200,10,<catboost.core.CatBoostRegressor object at 0x0...,"{'regressor__learning_rate': 0.09, 'regressor_...",1161.871863,749.994083,965.858116,973.878714,NaN,10


In [64]:
import matplotlib.pyplot as plt

In [65]:
%pip install plotly
import plotly.express as px
import plotly.graph_objects as go

Note: you may need to restart the kernel to use updated packages.


In [66]:
import seaborn as sns

In [67]:
sns.load_dataset("penguins")

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


In [68]:
dset = pd.concat([pd.Series(X_train.columns), 
                  pd.Series(grid.best_estimator_.named_steps["regressor"].feature_importances_)], axis= 1 )
dset = dset.set_axis(['attr', 'importance'], 
                     axis=1).sort_values(by='importance', 
                                                ascending=True)
fig = go.Figure(go.Bar(
            x=dset["importance"],
            y=dset['attr'],
            orientation='h'))

fig.update_layout(width=1500, height=500)

In [69]:
%%time
y_final = grid.best_estimator_.predict(X_test)

CPU times: total: 0 ns
Wall time: 4 ms


In [70]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_final, squared=False)

840.7404000988691

In [71]:
import joblib
joblib.dump(grid, "model.pkl")

['model.pkl']